# Scaling, sampling and modelling

In [52]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)

In [25]:
# Load cleaned dataset
df = pd.read_csv("Bank Leads Cleaned 20240206v2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68797 entries, 0 to 68796
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Gender                               68797 non-null  object 
 1   DOB                                  68797 non-null  object 
 2   Lead_Creation_Date                   68797 non-null  object 
 3   City_Code                            68797 non-null  object 
 4   City_Category                        68797 non-null  object 
 5   Employer_Category1                   68797 non-null  object 
 6   Employer_Category2                   68797 non-null  float64
 7   Monthly_Income                       68797 non-null  float64
 8   Customer_Existing_Primary_Bank_Code  68797 non-null  object 
 9   Primary_Bank_Type                    68797 non-null  object 
 10  Contacted                            68797 non-null  object 
 11  Source                      

In [26]:
df.describe()

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved,Year
count,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000,68797.000000
mean,3.735948,3599.859583,348.049336,35692.762766,3.933471,19.215630,1082.756741,3.989723,0.014754,1985.286844
std,0.786844,4810.412524,938.597201,24304.447313,0.912540,3.325489,670.955220,3.815124,0.120566,7.070241
min,1.000000,0.000000,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000,0.000000,1930.000000
25%,4.000000,1640.000000,0.000000,29000.000000,4.000000,19.213570,900.387994,0.000000,0.000000,1982.000000
50%,4.000000,2500.000000,0.000000,30000.000000,4.000000,19.213570,900.387994,2.000000,0.000000,1987.000000
75%,4.000000,4000.000000,350.000000,36000.000000,4.000000,19.213570,1102.922701,7.000000,0.000000,1990.000000
max,4.000000,97500.000000,42000.000000,300000.000000,6.000000,37.000000,13556.000000,10.000000,1.000000,1999.000000


## Feature engineering 

In [27]:
# Create debt to income ratio feature
def dti_optimized(data):
    # Vectorized calculation
    x = data['Existing_EMI'] / data['Monthly_Income']
    x[data['Existing_EMI'] == 0] = 0
    x[data['Monthly_Income'] == 0] = 1
    return x

df['Debt_Income'] = dti_optimized(df)

# Create Total Loan feature
def total_loan_optimized(data):
    # Ensure the calculation is vectorized
    return data['Loan_Amount'] * (1 + (data['Interest_Rate']/100)) ** data['Loan_Period']

df['Total_Loan'] = total_loan_optimized(df)

# Create Age feature - This operation is already efficient
df['Age'] = 2024 - df['Year']

# Drop unusable features - This operation is also fine
df.drop(['Lead_Creation_Date', 'DOB', 'City_Code', 'Customer_Existing_Primary_Bank_Code', 'Source'], axis=1, inplace=True)


## Categorical encoding 

In [28]:
df['Employer_Category2'] = df['Employer_Category2'].astype(str)

In [29]:
cat_feats = df[['Gender', 'City_Category', 'Employer_Category1', 'Employer_Category2', 'Primary_Bank_Type', 'Contacted', 'Source_Category']]

df = pd.get_dummies(df, columns=cat_feats.columns, drop_first=True, dtype=int)

In [30]:
df.drop(['Year'], axis=1, inplace=True)

In [31]:
df.drop(['Var1'], axis=1, inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68797 entries, 0 to 68796
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Monthly_Income          68797 non-null  float64
 1   Existing_EMI            68797 non-null  float64
 2   Loan_Amount             68797 non-null  float64
 3   Loan_Period             68797 non-null  float64
 4   Interest_Rate           68797 non-null  float64
 5   EMI                     68797 non-null  float64
 6   Approved                68797 non-null  int64  
 7   Debt_Income             68797 non-null  float64
 8   Total_Loan              68797 non-null  float64
 9   Age                     68797 non-null  int64  
 10  Gender_Male             68797 non-null  int32  
 11  City_Category_B         68797 non-null  int32  
 12  City_Category_C         68797 non-null  int32  
 13  Employer_Category1_B    68797 non-null  int32  
 14  Employer_Category1_C    68797 non-null

## Split the data 

In [33]:
seed = 42

X = df.drop(['Approved'], axis=1)
y = df['Approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

## Numerical scaling

In [34]:
numeric_feats = X_train[['Monthly_Income', 'Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI', 'Debt_Income', 'Total_Loan', 'Age']]
categorical_feats = X_train.drop(['Monthly_Income', 'Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI', 'Debt_Income', 'Total_Loan', 'Age'], axis=1)

# StandardScaler
ss = StandardScaler()
scaled = ss.fit_transform(numeric_feats)
scaled_df = pd.DataFrame(scaled, index=numeric_feats.index, columns=numeric_feats.columns)
X_train_ss = pd.concat([scaled_df, categorical_feats], axis=1)

# RobustScaler 
rs = RobustScaler()
scaled = rs.fit_transform(numeric_feats)
scaled_df = pd.DataFrame(scaled, index=numeric_feats.index, columns=numeric_feats.columns)
X_train_rs = pd.concat([scaled_df, categorical_feats], axis=1)

## Feature selection 

In [35]:
# RFE - ss
lr = LogisticRegression()

rfe = RFE(estimator=lr, n_features_to_select=10, step=1)

rfe.fit(X_train_ss, y_train)

# print("Feature ranking: ", rfe.ranking_)
# print("Feature selection: ", rfe.support_)

ss_rfe_table = pd.DataFrame({
    'Feature': X_train_ss.columns,
    'Ranking': rfe.ranking_,
    'SAelection': rfe.support_
})

ss_rfe_table.sort_values(by='Ranking')

c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

,Feature,Ranking,SAelection
12,Employer_Category1_B,1,True
1,Existing_EMI,1,True
21,Source_Category_D,1,True
4,Interest_Rate,1,True
8,Age,1,True
9,Gender_Male,1,True
18,Contacted_Y,1,True
11,City_Category_C,1,True
17,Primary_Bank_Type_P,1,True
13,Employer_Category1_C,1,True


In [36]:

selected_features_ss = list(ss_rfe_table[ss_rfe_table['Ranking'] == 1]['Feature'])

X_train_ss_rfe = X_train_ss[selected_features_ss]

In [37]:
# RFE - rs
lr = LogisticRegression()

rfe = RFE(estimator=lr, n_features_to_select=10, step=1)

rfe.fit(X_train_rs, y_train)

# print("Feature ranking: ", rfe.ranking_)
# print("Feature selection: ", rfe.support_)

rs_rfe_table = pd.DataFrame({
    'Feature': X_train_ss.columns,
    'Ranking': rfe.ranking_,
    'SAelection': rfe.support_
})

rs_rfe_table.sort_values(by='Ranking')

c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

,Feature,Ranking,SAelection
12,Employer_Category1_B,1,True
21,Source_Category_D,1,True
4,Interest_Rate,1,True
18,Contacted_Y,1,True
6,Debt_Income,1,True
17,Primary_Bank_Type_P,1,True
8,Age,1,True
9,Gender_Male,1,True
13,Employer_Category1_C,1,True
11,City_Category_C,1,True


In [38]:
selected_features_rs = list(rs_rfe_table[rs_rfe_table['Ranking'] == 1]['Feature'])

X_train_rs_rfe = X_train_rs[selected_features_rs]

## Sampling 

In [55]:
# SMOTE
smote = SMOTE(random_state=seed)

X_train_ss_rfe_smote, y_train_ss_smote = smote.fit_resample(X_train_ss_rfe, y_train)

print(f"Before SMOTE: {pd.Series(y_train).value_counts()}")
print(f"After SMOTE: {pd.Series(y_train_ss_smote).value_counts()}")

Before SMOTE: Approved
0    54223
1      814
Name: count, dtype: int64
After SMOTE: Approved
0    54223
1    54223
Name: count, dtype: int64


In [56]:
# SMOTE
smote = SMOTE(random_state=seed)

X_train_rs_rfe_smote, y_train_rs_smote = smote.fit_resample(X_train_rs_rfe, y_train)

print(f"Before SMOTE: {pd.Series(y_train).value_counts()}")
print(f"After SMOTE: {pd.Series(y_train_rs_smote).value_counts()}")

Before SMOTE: Approved
0    54223
1      814
Name: count, dtype: int64
After SMOTE: Approved
0    54223
1    54223
Name: count, dtype: int64


In [ ]:
# Random oversampling

In [ ]:
# Random undersampling

## Modelling 

In [50]:
# Logistic Regression | StandardScaler 
lr = LogisticRegression()

lr.fit(X_train_ss_rfe_smote, y_train_ss_smote)

y_preds_ss = lr.predict(X_test[selected_features_ss])

conf_matrix = confusion_matrix(y_test, y_preds_ss)
report = classification_report(y_test, y_preds_ss)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[5177 8382]
 [  23  178]]


Classification report...............
               precision    recall  f1-score   support

           0       1.00      0.38      0.55     13559
           1       0.02      0.89      0.04       201

    accuracy                           0.39     13760
   macro avg       0.51      0.63      0.30     13760
weighted avg       0.98      0.39      0.54     13760



In [51]:
# Logistic Regression | RobustScaler 
lr = LogisticRegression()

lr.fit(X_train_rs_rfe_smote, y_train_rs_smote)

y_preds_rs = lr.predict(X_test[selected_features_rs])

conf_matrix = confusion_matrix(y_test, y_preds_rs)
report = classification_report(y_test, y_preds_rs)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[    0 13559]
 [    0   201]]


Classification report...............
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     13559
           1       0.01      1.00      0.03       201

    accuracy                           0.01     13760
   macro avg       0.01      0.50      0.01     13760
weighted avg       0.00      0.01      0.00     13760



c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# MLP

In [53]:
# SVC | StandardScaler
svc = SVC(random_state=seed)

svc.fit(X_train_ss_rfe_smote, y_train_ss_smote)

conf_matrix = confusion_matrix(y_test, y_preds_ss)
report = classification_report(y_test, y_preds_ss)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[5177 8382]
 [  23  178]]


Classification report...............
               precision    recall  f1-score   support

           0       1.00      0.38      0.55     13559
           1       0.02      0.89      0.04       201

    accuracy                           0.39     13760
   macro avg       0.51      0.63      0.30     13760
weighted avg       0.98      0.39      0.54     13760



In [54]:
# SVC | RobustScaler
svc = SVC(random_state=seed)

svc.fit(X_train_rs_rfe_smote, y_train_rs_smote)

conf_matrix = confusion_matrix(y_test, y_preds_rs)
report = classification_report(y_test, y_preds_rs)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[    0 13559]
 [    0   201]]


Classification report...............
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     13559
           1       0.01      1.00      0.03       201

    accuracy                           0.01     13760
   macro avg       0.01      0.50      0.01     13760
weighted avg       0.00      0.01      0.00     13760



c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zive9\anaconda3\envs\iod\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Fine tuning

In [57]:
# Fine tuning | LogisticRegression, StandardScaler

# Logistic Regression | StandardScaler 
lr = LogisticRegression()

lr.fit(X_train_ss_rfe_smote, y_train_ss_smote)

y_preds_ss = lr.predict(X_test[selected_features_ss])

conf_matrix = confusion_matrix(y_test, y_preds_ss)
report = classification_report(y_test, y_preds_ss)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[5177 8382]
 [  23  178]]


Classification report...............
               precision    recall  f1-score   support

           0       1.00      0.38      0.55     13559
           1       0.02      0.89      0.04       201

    accuracy                           0.39     13760
   macro avg       0.51      0.63      0.30     13760
weighted avg       0.98      0.39      0.54     13760



In [61]:
lr = LogisticRegression()

lr.fit(X_train_ss_rfe_smote, y_train_ss_smote)

y_preds_ss = lr.predict(X_test[selected_features_ss])

probs = lr.predict_proba(X_test[selected_features_ss])

new_threshold = 0.2

y_pred_new_threshold = (probs[:, 1] > new_threshold).astype(int)

conf_matrix = confusion_matrix(y_test, y_pred_new_threshold)
report = classification_report(y_test, y_pred_new_threshold)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)

Confusion matrix.................
 [[ 2948 10611]
 [   10   191]]


Classification report...............
               precision    recall  f1-score   support

           0       1.00      0.22      0.36     13559
           1       0.02      0.95      0.03       201

    accuracy                           0.23     13760
   macro avg       0.51      0.58      0.20     13760
weighted avg       0.98      0.23      0.35     13760



In [66]:
# To make the threshold change persist in future predictions 

def predict_with_custom_threshold(model, X, threshold=0.5):
    probabilities = model.predict_proba(X)
    return (probabilities[:, 1] > threshold).astype(int)

# Usage
y_pred_custom = predict_with_custom_threshold(lr, X_test[selected_features_ss], 0.4)

conf_matrix = confusion_matrix(y_test, y_pred_custom)
report = classification_report(y_test, y_pred_custom)

print("Confusion matrix.................\n", conf_matrix)
print("\n\nClassification report...............\n", report)


Confusion matrix.................
 [[4670 8889]
 [  21  180]]


Classification report...............
               precision    recall  f1-score   support

           0       1.00      0.34      0.51     13559
           1       0.02      0.90      0.04       201

    accuracy                           0.35     13760
   macro avg       0.51      0.62      0.28     13760
weighted avg       0.98      0.35      0.50     13760

